In [13]:
import numpy as np
import cv2 as cv
import glob

In [14]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
rows = 7
cols = 7
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((rows*cols,3), np.float32)
objp[:,:2] = np.mgrid[0:cols,0:rows].T.reshape(-1,2)

# print(objp)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('./Images_bg/*.png')

for fname in images:
    img = cv.imread(fname)
    # img = cv.resize(img, (640, 480))
    print(fname)
    print(img.shape)
    h,w = img.shape[0], img.shape[1]
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # gray = cv.equalizeHist(gray)

    _, gray = cv.threshold(gray, 127, 255, cv.THRESH_BINARY)
    cv.imwrite(fname.replace(".png", "_gray.png").replace("Images_bg", "Grayscale"), gray)

    # Find the chessboard corners
    print(cv.checkChessboard(gray, (cols, rows)))
    # cv.imwrite(fname.replace(".jpg", "_gray.jpg"), gray)
    ret, corners = cv.findChessboardCornersSB(gray, (cols, rows), flags=0)
    print(ret)
    # print(corners.shape)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (cols, rows), corners2, ret)
        cv.imwrite(fname.replace(".png", "_new.png").replace("Images_bg", "Outputs"), img)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()

./Images_bg/5-removebg-preview.png
(577, 433, 3)
True
True


QObject::moveToThread: Current thread (0x3e30b20) is not the object's thread (0x3f0f6d0).
Cannot move to target thread (0x3e30b20)

QObject::moveToThread: Current thread (0x3e30b20) is not the object's thread (0x3f0f6d0).
Cannot move to target thread (0x3e30b20)

QObject::moveToThread: Current thread (0x3e30b20) is not the object's thread (0x3f0f6d0).
Cannot move to target thread (0x3e30b20)

QObject::moveToThread: Current thread (0x3e30b20) is not the object's thread (0x3f0f6d0).
Cannot move to target thread (0x3e30b20)

QObject::moveToThread: Current thread (0x3e30b20) is not the object's thread (0x3f0f6d0).
Cannot move to target thread (0x3e30b20)

QObject::moveToThread: Current thread (0x3e30b20) is not the object's thread (0x3f0f6d0).
Cannot move to target thread (0x3e30b20)

QObject::moveToThread: Current thread (0x3e30b20) is not the object's thread (0x3f0f6d0).
Cannot move to target thread (0x3e30b20)

QObject::moveToThread: Current thread (0x3e30b20) is not the object's thread

./Images_bg/1-removebg-preview.png
(577, 433, 3)
True
True
./Images_bg/7-removebg-preview.png
(577, 433, 3)
True
True
./Images_bg/9-removebg-preview.png
(577, 433, 3)
True
True
./Images_bg/3-removebg-preview.png
(577, 433, 3)
True
True
./Images_bg/4-removebg-preview.png
(577, 433, 3)
True
True


In [19]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

rotation_matrices = []

# Iterate through each rotation vector and convert to a rotation matrix
for rvec in rvecs:
    rotation_matrix, _ = cv.Rodrigues(rvec)
    rotation_matrices.append(rotation_matrix)

In [20]:
print(ret)
print("-------------------------------------------")
print(mtx)
print("-------------------------------------------")
print(dist)
print("-------------------------------------------")
print(rotation_matrices)
print("-------------------------------------------")
print(tvecs)
print("-------------------------------------------")

print(newcameramtx)
print("-------------------------------------------")
print(roi)
print("-------------------------------------------")

0.9149182751917811
-------------------------------------------
[[440.45051995   0.         228.24297229]
 [  0.         439.22568069 261.16194565]
 [  0.           0.           1.        ]]
-------------------------------------------
[[ 9.82635927e-03  5.86473613e-01 -5.65289782e-03 -8.69117864e-04
  -1.31064782e+00]]
-------------------------------------------
[array([[ 0.81490218,  0.57953592,  0.00851819],
       [-0.57959034,  0.81472466,  0.01728477],
       [ 0.00307717, -0.01902246,  0.99981432]]), array([[ 0.99965714,  0.02157436,  0.01483713],
       [-0.02188315,  0.9995405 ,  0.02097425],
       [-0.01437781, -0.02129174,  0.99966992]]), array([[ 0.7495146 , -0.5075379 , -0.42500958],
       [ 0.6285774 ,  0.7470276 ,  0.21642599],
       [ 0.2076495 , -0.42936586,  0.87893495]]), array([[ 9.64428090e-01, -2.63496256e-01,  2.11703303e-02],
       [ 2.63566413e-01,  9.64641099e-01, -5.44838649e-04],
       [-2.02782078e-02,  6.10524573e-03,  9.99775735e-01]]), array([[ 0.9989

In [23]:
# undistort
img = cv.imread('./Images_bg/7-removebg-preview.png')
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

True